对于sm（ssd-mobilenet）的inference代码，tensorboard，或产生ckpt的过程，详见
https://www.cnblogs.com/White-xzx/p/9503203.html

对于label.pbtxt的生成，详见
https://www.jianshu.com/p/a8e8ff37a0bc

当然，tensorflow官方github上也是应有尽有的（object detection API）

以下是对于tod（tensorflow object detection）的一些总结：
1. 教程写的非常详细，网上也可以找到足够的代码。tf官方写的demo.ipynb必须要求tf2，但其他（比如所有训练代码）都要求tf1.15以及numpy1.17
2. 官方推出的sm预训练模型在视频以及大部分图片上跑的效果是客观的，视频在cpu上可以达到15fps，当然也有结果比较糟糕的图片（见下图）




如果想要用自己的数据集fine tune图像检测模型的话（大部分应用都是如此）
1. 需要一个自己的数据集，格式最好跟coco数据集差不多。tf官方写的数据预处理包括了coco，pets数据集等，所以如果数据集完全跟coco一样的话只需要跑

python create_coco_tf_record.py --train_image_dir /coco/train2017 --val_image_dir /coco/val2017 --test_image_dir /coco/test2017 --train_annotations_file /coco/annotations/instances_train2017.json --val_annotations_file /coco/annotations/instances_val2017.json --testdev_annotations_file /coco/image_info_test-dev2017.json

其中需要注意的是coco数据集除了常有的图片annotation，另附带图片height，width等数值，如果自己的数据集里没有的话就不能直接用官方的coco预处理代码

2. （以下以FLIR的红外数据集为例：标注与coco基本相同，但需要手动加入每张图的height和width就可以借用官方代码了）
详见create_flir_tf_record.py以及手动读取的train.json, val.json, test.json

3. 这样我们就有了做训练需要的tfrecord。下一步是生成label.pbtxt，其实就是把每个物品标个号。
详见untitled.ipynb

4. 最后一步，在网上下载一个官方的预训练模型（我选择的是sm_v1_coco.ckpt），以及一个对应的config文件（ssdlitem_v1_coco.config)。
    1. 修改line 9，总标注个数（FLIR是80，coco是90）
    2. line 156，训练从哪里开始，就用之前下载的ckpt就好了
    3. line 175 & 189，train & val 的 tfrecord
    4. line 177 & 191，使用第三步生成的pbtxt文件
    5. 还可以选择修改：141（batch size），145（learning rate），162（epochs/steps），181（eval samples）
    
    
5. 现在使用的大部分代码还是官方提供的，后面会尝试有一些自己的创新

